In [1]:
import pandas as pd

from SMILESX import main, inference
%matplotlib inline

Using TensorFlow backend.


#### Definition of data

In [2]:
validation_data_dir = "./data/"

In [3]:
extension = '.csv'

In [4]:
data_name = 'FreeSolv' # FreeSolv, ESOL, Lipophilicity
prop_tag = ''

In [5]:
if data_name == 'FreeSolv':
    data_filename = 'FreeSolv_SAMPL'
    prop_tag = 'expt'
elif data_name == 'ESOL':
    data_filename = 'ESOL_delaney-processed'
    prop_tag = 'measured log solubility in mols per litre'
elif data_name == 'Lipophilicity':
    data_filename = 'Lipophilicity'
    prop_tag = 'exp'
else:
    data_filename = data_name
    prop_tag = prop_tag

In [6]:
sol_data = pd.read_csv(validation_data_dir+data_filename+extension)

In [7]:
sol_data.head(3)

,Unnamed: 0,iupac,smiles,expt,calc
0,0,"4-methoxy-N,N-dimethyl-benzamide",COc1ccc(C(=O)N(C)C)cc1,-11.01,-9.625
1,1,methanesulfonyl chloride,CS(=O)(=O)Cl,-4.87,-6.219
2,2,3-methylbut-1-ene,C=CC(C)C,1.83,2.452


In [8]:
sol_data = sol_data[['smiles',prop_tag]]

In [9]:
sol_data.head()

,smiles,expt
0,COc1ccc(C(=O)N(C)C)cc1,-11.01
1,CS(=O)(=O)Cl,-4.87
2,C=CC(C)C,1.83
3,CCc1cnccn1,-5.45
4,CCCCCCCO,-4.21


In [10]:
sol_data.shape

(642, 2)

#### Hyperparameters optimization with GPyOpt (Bayesian optimization)

In [11]:
dhyp_range = [int(2**itn) for itn in range(3,11)] # 
dalpha_range = [float(ialpha/10.) for ialpha in range(20,40,1)] # Adam's learning rate = 10^(-dalpha_range)

if data_name != 'Lipophilicity':
    bounds = [
        {'name': 'lstmunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'denseunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'embedding', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'batchsize', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'lrate', 'type': 'discrete', 'domain': dalpha_range}
    ]
else:
    bounds = [
        {'name': 'lstmunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'denseunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'embedding', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'batchsize', 'type': 'discrete', 'domain': (1024, 1024)}, # fixed
        {'name': 'lrate', 'type': 'discrete', 'domain': (3, 3)} # fixed
    ]

In [12]:
main.Main(data=sol_data, 
                   data_name=data_name, 
                   data_units='', 
                   bayopt_bounds=bounds, 
                   k_fold_number = 3, 
                   augmentation = True, 
                   outdir = "./data/", 
                   bayopt_n_epochs = 1,
                   bayopt_n_rounds = 1, 
                   bayopt_it_factor = 1, 
                   bayopt_on = True, 
                   lstmunits_ref = 128, 
                   denseunits_ref = 16, 
                   embedding_ref = 32, 
                   n_gpus = 1, 
                   bridge_type = 'NVLink', 
                   batch_size_ref = 8,
                   alpha_ref = 22, 
                   patience = 50, 
                   n_epochs = 100)

***SMILES_X starts...***


******
***Fold #0 initiated...***
******
***Sampling and splitting of the dataset.***

Scaler: RobustScaler(copy=True, quantile_range=(5.0, 95.0), with_centering=True,
             with_scaling=True)
Train/valid/test splits: 0.80/0.10/0.10


***Data augmentation to True***

Enumerated SMILES:
	Training set: 4285
	Validation set: 501
	Test set: 569

***Tokenization of SMILES.***

Examples of tokenized SMILES from a training set:
[[' ', 'C', 'Cl', ' '], [' ', 'Cl', 'C', ' '], [' ', 'C', 'C', 'c', '1', 'c', 'c', 'n', 'c', 'c', '1', ' '], [' ', 'C', '(', 'c', '1', 'c', 'c', 'n', 'c', 'c', '1', ')', 'C', ' '], [' ', 'c', '1', '(', 'C', 'C', ')', 'c', 'c', 'n', 'c', 'c', '1', ' ']]

Number of tokens only present in a training set: 32

Number of tokens only present in a validation set: 27
Is the validation set a subset of the training set: True
What are the tokens by which they differ: set()

Number of tokens only present in a test set: 27
Is the test set a subset o

Epoch 21/100
134/134 [==============================] - 1s 7ms/step - loss: 0.0103 - mean_absolute_error: 0.0782 - mean_squared_error: 0.0103 - val_loss: 0.0094 - val_mean_absolute_error: 0.0771 - val_mean_squared_error: 0.0094
Epoch 22/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0090 - mean_absolute_error: 0.0719 - mean_squared_error: 0.0090 - val_loss: 0.0111 - val_mean_absolute_error: 0.0881 - val_mean_squared_error: 0.0111
Epoch 23/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0093 - mean_absolute_error: 0.0730 - mean_squared_error: 0.0093 - val_loss: 0.0110 - val_mean_absolute_error: 0.0823 - val_mean_squared_error: 0.0110
Epoch 24/100
134/134 [==============================] - 1s 8ms/step - loss: 0.0090 - mean_absolute_error: 0.0726 - mean_squared_error: 0.0090 - val_loss: 0.0098 - val_mean_absolute_error: 0.0780 - val_mean_squared_error: 0.0098
Epoch 25/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0083 - mea

Epoch 57/100
134/134 [==============================] - 1s 7ms/step - loss: 0.0056 - mean_absolute_error: 0.0560 - mean_squared_error: 0.0056 - val_loss: 0.0106 - val_mean_absolute_error: 0.0783 - val_mean_squared_error: 0.0106
Epoch 58/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0059 - mean_absolute_error: 0.0576 - mean_squared_error: 0.0059 - val_loss: 0.0078 - val_mean_absolute_error: 0.0646 - val_mean_squared_error: 0.0078
Epoch 59/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0060 - mean_absolute_error: 0.0580 - mean_squared_error: 0.0060 - val_loss: 0.0080 - val_mean_absolute_error: 0.0671 - val_mean_squared_error: 0.0080
Epoch 60/100
134/134 [==============================] - 1s 7ms/step - loss: 0.0052 - mean_absolute_error: 0.0546 - mean_squared_error: 0.0052 - val_loss: 0.0081 - val_mean_absolute_error: 0.0672 - val_mean_squared_error: 0.0081
Epoch 61/100
134/134 [==============================] - 1s 8ms/step - loss: 0.0054 - mea

Epoch 93/100
134/134 [==============================] - 1s 8ms/step - loss: 0.0048 - mean_absolute_error: 0.0521 - mean_squared_error: 0.0048 - val_loss: 0.0082 - val_mean_absolute_error: 0.0668 - val_mean_squared_error: 0.0082
Epoch 94/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0044 - mean_absolute_error: 0.0485 - mean_squared_error: 0.0044 - val_loss: 0.0087 - val_mean_absolute_error: 0.0686 - val_mean_squared_error: 0.0087
Epoch 95/100
134/134 [==============================] - 1s 8ms/step - loss: 0.0043 - mean_absolute_error: 0.0494 - mean_squared_error: 0.0043 - val_loss: 0.0103 - val_mean_absolute_error: 0.0753 - val_mean_squared_error: 0.0103
Epoch 96/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0041 - mean_absolute_error: 0.0481 - mean_squared_error: 0.0041 - val_loss: 0.0091 - val_mean_absolute_error: 0.0676 - val_mean_squared_error: 0.0091
Epoch 97/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0042 - mea

9/9 [==============================] - 0s 18ms/step - loss: 0.0267 - mean_absolute_error: 0.1321 - mean_squared_error: 0.0267 - val_loss: 0.0304 - val_mean_absolute_error: 0.1424 - val_mean_squared_error: 0.0304
Epoch 13/100
9/9 [==============================] - 0s 19ms/step - loss: 0.0268 - mean_absolute_error: 0.1311 - mean_squared_error: 0.0268 - val_loss: 0.0263 - val_mean_absolute_error: 0.1373 - val_mean_squared_error: 0.0263
Epoch 14/100
9/9 [==============================] - 0s 11ms/step - loss: 0.0270 - mean_absolute_error: 0.1309 - mean_squared_error: 0.0270 - val_loss: 0.0325 - val_mean_absolute_error: 0.1483 - val_mean_squared_error: 0.0325
Epoch 15/100
9/9 [==============================] - 0s 12ms/step - loss: 0.0249 - mean_absolute_error: 0.1278 - mean_squared_error: 0.0249 - val_loss: 0.0263 - val_mean_absolute_error: 0.1364 - val_mean_squared_error: 0.0263
Epoch 16/100
9/9 [==============================] - 0s 12ms/step - loss: 0.0240 - mean_absolute_error: 0.1265 - m

9/9 [==============================] - 0s 24ms/step - loss: 0.0182 - mean_absolute_error: 0.1099 - mean_squared_error: 0.0182 - val_loss: 0.0203 - val_mean_absolute_error: 0.1160 - val_mean_squared_error: 0.0203
Epoch 49/100
9/9 [==============================] - 0s 20ms/step - loss: 0.0182 - mean_absolute_error: 0.1088 - mean_squared_error: 0.0182 - val_loss: 0.0202 - val_mean_absolute_error: 0.1122 - val_mean_squared_error: 0.0202
Epoch 50/100
9/9 [==============================] - 0s 25ms/step - loss: 0.0173 - mean_absolute_error: 0.1066 - mean_squared_error: 0.0173 - val_loss: 0.0200 - val_mean_absolute_error: 0.1147 - val_mean_squared_error: 0.0200
Epoch 51/100
9/9 [==============================] - 0s 21ms/step - loss: 0.0176 - mean_absolute_error: 0.1082 - mean_squared_error: 0.0176 - val_loss: 0.0244 - val_mean_absolute_error: 0.1258 - val_mean_squared_error: 0.0244
Epoch 52/100
9/9 [==============================] - 0s 24ms/step - loss: 0.0177 - mean_absolute_error: 0.1075 - m

9/9 [==============================] - 0s 21ms/step - loss: 0.0105 - mean_absolute_error: 0.0787 - mean_squared_error: 0.0105 - val_loss: 0.0214 - val_mean_absolute_error: 0.1112 - val_mean_squared_error: 0.0214
Epoch 85/100
9/9 [==============================] - 0s 24ms/step - loss: 0.0101 - mean_absolute_error: 0.0784 - mean_squared_error: 0.0101 - val_loss: 0.0190 - val_mean_absolute_error: 0.1063 - val_mean_squared_error: 0.0190
Epoch 86/100
9/9 [==============================] - 0s 23ms/step - loss: 0.0100 - mean_absolute_error: 0.0779 - mean_squared_error: 0.0100 - val_loss: 0.0177 - val_mean_absolute_error: 0.0982 - val_mean_squared_error: 0.0177
Epoch 87/100
9/9 [==============================] - 0s 21ms/step - loss: 0.0098 - mean_absolute_error: 0.0770 - mean_squared_error: 0.0098 - val_loss: 0.0208 - val_mean_absolute_error: 0.1070 - val_mean_squared_error: 0.0208
Epoch 88/100
9/9 [==============================] - 0s 23ms/step - loss: 0.0098 - mean_absolute_error: 0.0766 - m

538/538 [==============================] - 4s 7ms/step - loss: 0.0230 - mean_absolute_error: 0.1172 - mean_squared_error: 0.0230 - val_loss: 0.0248 - val_mean_absolute_error: 0.1065 - val_mean_squared_error: 0.0248
Epoch 5/100
538/538 [==============================] - 3s 5ms/step - loss: 0.0184 - mean_absolute_error: 0.1034 - mean_squared_error: 0.0184 - val_loss: 0.0296 - val_mean_absolute_error: 0.1063 - val_mean_squared_error: 0.0296
Epoch 6/100
538/538 [==============================] - 4s 8ms/step - loss: 0.0167 - mean_absolute_error: 0.1001 - mean_squared_error: 0.0167 - val_loss: 0.0253 - val_mean_absolute_error: 0.1059 - val_mean_squared_error: 0.0253
Epoch 7/100
538/538 [==============================] - 3s 6ms/step - loss: 0.0163 - mean_absolute_error: 0.0975 - mean_squared_error: 0.0163 - val_loss: 0.0331 - val_mean_absolute_error: 0.1109 - val_mean_squared_error: 0.0331
Epoch 8/100
538/538 [==============================] - 4s 8ms/step - loss: 0.0141 - mean_absolute_error:

538/538 [==============================] - 3s 5ms/step - loss: 0.0029 - mean_absolute_error: 0.0398 - mean_squared_error: 0.0029 - val_loss: 0.0158 - val_mean_absolute_error: 0.0735 - val_mean_squared_error: 0.0158
Epoch 41/100
538/538 [==============================] - 3s 5ms/step - loss: 0.0024 - mean_absolute_error: 0.0366 - mean_squared_error: 0.0024 - val_loss: 0.0152 - val_mean_absolute_error: 0.0713 - val_mean_squared_error: 0.0152
Epoch 42/100
538/538 [==============================] - 3s 6ms/step - loss: 0.0034 - mean_absolute_error: 0.0431 - mean_squared_error: 0.0034 - val_loss: 0.0131 - val_mean_absolute_error: 0.0708 - val_mean_squared_error: 0.0131
Epoch 43/100
538/538 [==============================] - 3s 6ms/step - loss: 0.0033 - mean_absolute_error: 0.0427 - mean_squared_error: 0.0033 - val_loss: 0.0144 - val_mean_absolute_error: 0.0721 - val_mean_squared_error: 0.0144
Epoch 44/100
538/538 [==============================] - 3s 6ms/step - loss: 0.0027 - mean_absolute_er

Epoch 76/100
538/538 [==============================] - 3s 6ms/step - loss: 0.0010 - mean_absolute_error: 0.0246 - mean_squared_error: 0.0010 - val_loss: 0.0150 - val_mean_absolute_error: 0.0745 - val_mean_squared_error: 0.0150
Epoch 77/100
538/538 [==============================] - 4s 8ms/step - loss: 0.0018 - mean_absolute_error: 0.0324 - mean_squared_error: 0.0018 - val_loss: 0.0132 - val_mean_absolute_error: 0.0688 - val_mean_squared_error: 0.0132
Epoch 78/100
538/538 [==============================] - 5s 8ms/step - loss: 0.0013 - mean_absolute_error: 0.0275 - mean_squared_error: 0.0013 - val_loss: 0.0129 - val_mean_absolute_error: 0.0690 - val_mean_squared_error: 0.0129
Epoch 79/100
538/538 [==============================] - 4s 8ms/step - loss: 0.0012 - mean_absolute_error: 0.0257 - mean_squared_error: 0.0012 - val_loss: 0.0139 - val_mean_absolute_error: 0.0685 - val_mean_squared_error: 0.0139
Epoch 80/100
538/538 [==============================] - 4s 7ms/step - loss: 0.0014 - mea

In [13]:
pred_from_ens = inference.Inference(data_name=data_name, 
                                        smiles_list = ['CC','CCC','C=O','ABC','DEF'], 
                                        data_units = '',
                                        k_fold_number = 3,
                                        augmentation = True, 
                                        outdir = "./data/")

***SMILES_X for inference starts...***


***Checking the SMILES list for inference***

***Data augmentation.***

Enumerated SMILES: 5

***Tokenization of SMILES.***

Full vocabulary: ['pad', 'unk', 'C', '1', 'F', 'c', 'S', '4', '[N+]', '#', 'n', '[nH]', '2', '\\', ')', '=', '5', 's', '-', '[O-]', '3', 'Br', '/', '(', '[C@@]', 'N', '[C@]', '[C@H]', 'P', '[C@@H]', 'Cl', 'I', ' ', '[S+2]', 'O']
Of size: 35

Maximum length of tokenized SMILES: 51 tokens

***Inference of SMILES property done.***


In [14]:
pred_from_ens

,SMILES,ens_pred_mean,ens_pred_sd
0,CC,0.386122,0.0290024
1,CCC,0.41768,0.0214713
2,C=O,0.00934574,0.0404441
